In [1]:
print(4)

4


In [28]:
#import
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
#from scipy import stats
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import PowerTransformer
#all metrics
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, roc_curve, auc
#random forest
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
# import StratifiedKFold
from sklearn.model_selection import StratifiedKFold

#IMPORTANT
from ..library.tally import *


ImportError: attempted relative import with no known parent package

In [29]:
#load data
df_train=pd.read_csv('../data/for_modelling/credit_score_train.csv')
df_val=pd.read_csv('../data/for_modelling/credit_score_valid.csv')

#split X from y
X_train, y_train = split_X_from_y(df_train)
X_val, y_val = split_X_from_y(df_val)

s=create_tally(X_train, X_val, y_train, y_val, [RandomForestClassifier(random_state=42), GradientBoostingClassifier(random_state=42) ], ['RandomForestClassifier', 'GradientBoostingClassifier'])
s

/opt/homebrew/lib/python3.11/site-packages/numpy/core/_methods.py:239: RuntimeWarning: overflow encountered in multiply
  x = um.multiply(x, x, out=x)
/opt/homebrew/lib/python3.11/site-packages/numpy/core/_methods.py:250: RuntimeWarning: overflow encountered in reduce
  ret = umr_sum(x, axis, dtype, out, keepdims=keepdims, where=where)
/opt/homebrew/lib/python3.11/site-packages/numpy/core/_methods.py:239: RuntimeWarning: overflow encountered in multiply
  x = um.multiply(x, x, out=x)
/opt/homebrew/lib/python3.11/site-packages/numpy/core/_methods.py:239: RuntimeWarning: overflow encountered in multiply
  x = um.multiply(x, x, out=x)
/opt/homebrew/lib/python3.11/site-packages/numpy/core/_methods.py:250: RuntimeWarning: overflow encountered in reduce
  ret = umr_sum(x, axis, dtype, out, keepdims=keepdims, where=where)
/opt/homebrew/lib/python3.11/site-packages/numpy/core/_methods.py:239: RuntimeWarning: overflow encountered in multiply
  x = um.multiply(x, x, out=x)


,model,accuracy,accuracy_std,precision,precision_std,recall,recall_std,f1,f1_std
0,RandomForestClassifier,0.710095,0.042361,0.488819,0.208687,0.173968,0.069293,0.254133,0.100798
0,GradientBoostingClassifier,0.708584,0.035948,0.457031,0.142007,0.219206,0.090353,0.294851,0.110176


In [ ]:
#visualize s with std


